In [1]:
import numpy as np
import pandas as pd

df=pd.read_csv('clean_data.csv')
df.head()

,Text,Emotion,Clean_text
0,i didnt feel humiliated,sadness,didnt feel humiliated
1,i can go from feeling so hopeless to so damned...,sadness,feeling hopeless damned hopeful cares awake
2,im grabbing a minute to post i feel greedy wrong,anger,im grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,love,feeling nostalgic fireplace know property
4,i am feeling grouchy,anger,feeling grouchy


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Conv1D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

In [3]:
voc_size=10000
onehot_repr=[one_hot(words,voc_size)for words in df['Clean_text']] 
print(onehot_repr)

[[6461, 1729, 8365], [3640, 6819, 9562, 2046, 5492, 8656], [796, 3950, 4188, 8626, 1729, 7400, 9443], [3640, 963, 5066, 4544, 4973], [3640, 6601], [3538, 3640, 7291, 9869, 8540, 5319, 6725], [3538, 4949, 7682, 614, 4135, 3538, 9173, 7247, 7502, 8276, 1729, 7119, 9081], [1729, 8993, 7579, 8714, 2349, 7034, 3948, 3133], [4112, 9157, 1729, 4112, 110, 5842, 1395], [1729, 670], [1729, 7119, 3951, 9596, 6925], [1729, 8048, 7900, 7690, 4925, 488, 3551, 5108], [9696, 8376, 4620, 7034, 1729, 5898], [1729, 629, 9116, 4919], [981, 8107, 2987, 5417, 6524, 28, 238, 2692, 2914, 4620, 3170, 3662, 7579, 1551, 4829, 1881, 7526, 2321, 2803, 7291, 5058], [1729, 3979, 3603], [6461, 1729, 8087], [1729, 8045, 2809, 4620], [7221, 3640, 9302, 6132, 3458, 6920, 398, 3456, 7682, 6132, 4570], [1729, 8739, 2893, 1828, 5067, 3203], [1729, 2726, 776, 8355], [3640, 950, 3789, 8437, 3636, 1729, 9495, 9395, 284, 1320, 1934, 2572, 7943, 4244, 6203, 8636, 1244], [3640, 6595, 9297], [1866, 3636, 4659, 5683, 6116, 2827, 8

In [4]:
type(onehot_repr)

list

In [5]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 6461 1729 8365]
 [   0    0    0 ... 2046 5492 8656]
 [   0    0    0 ... 1729 7400 9443]
 ...
 [   0    0    0 ... 1290 2726 7267]
 [   0    0    0 ... 3626  933 2868]
 [   0    0    0 ...    0  933 6538]]


In [6]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Conv1D(100, kernel_size=3, activation = "relu"))
model1.add(Bidirectional(LSTM(150, return_sequences=True)))
model1.add(Dropout(0.5))
model1.add(Bidirectional(LSTM(50)))
model1.add(Dropout(0.2))
model1.add(Dense(50, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(50, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Flatten())
model1.add(Dense(150, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(6,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=tf.compat.v1.train.AdamOptimizer(
    learning_rate=0.00001, beta1=0.9, beta2=0.999, epsilon=10e-08, use_locking=False,
    name='Adam'),metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
conv1d (Conv1D)              (None, 18, 100)           12100     
_________________________________________________________________
bidirectional (Bidirectional (None, 18, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 18, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               140400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5

In [7]:
dummy=pd.get_dummies(df['Emotion'])
df=pd.concat([df, dummy], axis=1)
df.drop('Emotion', axis=1, inplace=True)
df.head()

,Text,Clean_text,anger,fear,happy,love,sadness,surprise
0,i didnt feel humiliated,didnt feel humiliated,0,0,0,0,1,0
1,i can go from feeling so hopeless to so damned...,feeling hopeless damned hopeful cares awake,0,0,0,0,1,0
2,im grabbing a minute to post i feel greedy wrong,im grabbing minute post feel greedy wrong,1,0,0,0,0,0
3,i am ever feeling nostalgic about the fireplac...,feeling nostalgic fireplace know property,0,0,0,1,0,0
4,i am feeling grouchy,feeling grouchy,1,0,0,0,0,0


In [8]:
y=df[['anger', 'fear', 'happy', 'love', 'sadness', 'surprise']]
len(embedded_docs),y.shape

(21459, (21459, 6))

In [9]:
type(y)

pandas.core.frame.DataFrame

In [10]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [11]:
X_final.shape,y_final.shape

((21459, 20), (21459, 6))

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42)

In [13]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100, batch_size=64)

Epoch 1/100
252/252 [==============================] - 10s 24ms/step - loss: 2.5257 - accuracy: 0.2703 - val_loss: 2.5046 - val_accuracy: 0.3197
Epoch 2/100
252/252 [==============================] - 5s 21ms/step - loss: 2.4799 - accuracy: 0.3000 - val_loss: 2.4385 - val_accuracy: 0.3197
Epoch 3/100
252/252 [==============================] - 5s 22ms/step - loss: 2.3843 - accuracy: 0.3019 - val_loss: 2.2934 - val_accuracy: 0.3197
Epoch 4/100
252/252 [==============================] - 5s 21ms/step - loss: 2.3221 - accuracy: 0.3103 - val_loss: 2.2575 - val_accuracy: 0.3197
Epoch 5/100
252/252 [==============================] - 5s 21ms/step - loss: 2.2884 - accuracy: 0.3161 - val_loss: 2.2314 - val_accuracy: 0.3197
Epoch 6/100
252/252 [==============================] - 5s 21ms/step - loss: 2.2561 - accuracy: 0.3161 - val_loss: 2.2035 - val_accuracy: 0.3197
Epoch 7/100
252/252 [==============================] - 5s 21ms/step - loss: 2.2323 - accuracy: 0.3127 - val_loss: 2.1802 - val_accuracy

252/252 [==============================] - 5s 21ms/step - loss: 1.0680 - accuracy: 0.6346 - val_loss: 1.5046 - val_accuracy: 0.5226
Epoch 58/100
252/252 [==============================] - 5s 21ms/step - loss: 1.0571 - accuracy: 0.6415 - val_loss: 1.5033 - val_accuracy: 0.5143
Epoch 59/100
252/252 [==============================] - 5s 21ms/step - loss: 1.0467 - accuracy: 0.6422 - val_loss: 1.5077 - val_accuracy: 0.5262
Epoch 60/100
252/252 [==============================] - 5s 21ms/step - loss: 1.0400 - accuracy: 0.6433 - val_loss: 1.5144 - val_accuracy: 0.5303
Epoch 61/100
252/252 [==============================] - 5s 21ms/step - loss: 1.0273 - accuracy: 0.6504 - val_loss: 1.5099 - val_accuracy: 0.5273
Epoch 62/100
252/252 [==============================] - 5s 21ms/step - loss: 1.0241 - accuracy: 0.6511 - val_loss: 1.5143 - val_accuracy: 0.5303
Epoch 63/100
252/252 [==============================] - 5s 21ms/step - loss: 1.0101 - accuracy: 0.6571 - val_loss: 1.5089 - val_accuracy: 0.526